In [1]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import os

import models.dcgan as dcgan
import models.mlp as mlp


In [14]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
        print(m)
        print("'W' mean: {0}, std: {1}".format(
            m.weight.data.mean(), m.weight.data.std()))
        print()
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
        print(m)
        print("'W' mean: {0}, std: {1}".format(
            m.weight.data.mean(), m.weight.data.std()))
        print("'b' mean: {0}, std: {1}".format(
            m.bias.data.mean(), m.bias.data.std()))
        print()
    else:
        print(classname)
        print()

In [15]:
netG = dcgan.DCGAN_G(64, 100, 3, 64, 1, 0)

In [16]:
netG.apply(weights_init)

ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
'W' mean: -9.007890184875578e-06, std: 0.02000388689339161

BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
'W' mean: 1.00065279006958, std: 0.02039995975792408
'b' mean: 0.0, std: 0.0

ReLU

ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
'W' mean: 1.1788062693085521e-05, std: 0.020006170496344566

BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
'W' mean: 0.9978319406509399, std: 0.01966889761388302
'b' mean: 0.0, std: 0.0

ReLU

ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
'W' mean: 2.4888087864383124e-05, std: 0.01996551640331745

BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
'W' mean: 0.997511088848114, std: 0.020087389275431633
'b' mean: 0.0, std: 0.0

ReLU

ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(

DCGAN_G(
  (main): Sequential(
    (initial_100-512_convt): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (initial_512_batchnorm): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (initial_512_relu): ReLU(inplace)
    (pyramid_512-256_convt): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid_256_batchnorm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pyramid_256_relu): ReLU(inplace)
    (pyramid_256-128_convt): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid_128_batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pyramid_128_relu): ReLU(inplace)
    (pyramid_128-64_convt): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid_64_batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, af

In [19]:
netD = dcgan.DCGAN_D(64, 100, 3, 64, 1, 0)

In [20]:
netD.apply(weights_init)

Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
'W' mean: -0.0009491209057159722, std: 0.02020801045000553

LeakyReLU

Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
'W' mean: -6.40585640212521e-05, std: 0.020035505294799805

BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
'W' mean: 0.9989120960235596, std: 0.020514201372861862
'b' mean: 0.0, std: 0.0

LeakyReLU

Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
'W' mean: -3.122820271528326e-05, std: 0.019969118759036064

BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
'W' mean: 1.0002846717834473, std: 0.02025088667869568
'b' mean: 0.0, std: 0.0

LeakyReLU

Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
'W' mean: 1.9230408724979497e-05, std: 0.01998784765601158

BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
'W' me

DCGAN_D(
  (main): Sequential(
    (initial_conv_3-64): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (initial_relu_64): LeakyReLU(negative_slope=0.2, inplace)
    (pyramid_64-128_conv): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid_128_batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pyramid_128_relu): LeakyReLU(negative_slope=0.2, inplace)
    (pyramid_128-256_conv): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid_256_batchnorm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pyramid_256_relu): LeakyReLU(negative_slope=0.2, inplace)
    (pyramid_256-512_conv): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid_512_batchnorm): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pyramid_512_relu): Lea